In [19]:
import pandas as pd
#import the library we use to open URLs
import urllib.request

In [20]:
#identify the site page to scrape
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [21]:
#open teh url using teh request and put the page into a variable
page = urllib.request.urlopen(url)

In [22]:
#import BeautifulSoup library so we can parse HTML documents
! pip install BeautifulSoup4
! pip install lxml
! pip install html5lib


In [23]:
from bs4 import BeautifulSoup

In [24]:
#parse the HTML from the URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page,"html.parser")

In [25]:
#find the table
all_tables = soup.find('table')


In [26]:
#create an object for table
main_table= soup.find('table', class_='wikitable sortable')


In [27]:
#loop through rows
A=[]
B=[]
C=[]


for row in main_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
     

In [28]:
#build dataframe
df = pd.DataFrame (A, columns=['Postal Code'])
df['Borough'] = B
df['Neighborhood'] = C
df

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


In [29]:
#Format dataframe
left = df['Postal Code'].str[:3]
df['Postal Code'] = left
left1 = df['Borough'].str.split('\n').str[0]
df['Borough'] = left1
left2 = df['Neighborhood'].str.split('\n').str[0]
df['Neighborhood'] = left2
df.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [71]:
#remove not assigned Borough's and reset index
df2 = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [98]:
#Groupby Postal Code
df3 = df2.groupby('Postal Code')
df3.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [190]:
#import coordinates
fileN = ('Geospatial_Coordinates.csv')
dfile = pd.read_csv(fileN)
dfile['Postal Code'] = dfile['Postal Code']

Combine Tables

In [193]:
merged_left = pd.merge(left=df2, right=dfile, how='left', left_on = 'Postal Code', right_on='Postal Code')
merged_left.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
